In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import math
%matplotlib inline
import matplotlib as plt
from sklearn.pipeline import Pipeline


In [2]:
negative_with_seq = pd.read_csv("./data/negative_examples.csv")
positive_with_seq = pd.read_csv("./data/positive_examples.csv")


In [3]:
n = positive_with_seq["seq_new"].apply(lambda x: True if "N" in x else False)
np.where(n == True)


(array([ 15218,  15223,  15224, ..., 153676, 153677, 153678]),)

In [4]:
bases = dict(zip("ACGTN", [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]))
bases 




{'A': [1, 0, 0, 0, 0],
 'C': [0, 1, 0, 0, 0],
 'G': [0, 0, 1, 0, 0],
 'T': [0, 0, 0, 1, 0],
 'N': [0, 0, 0, 0, 1]}

In [5]:
def transform(df):
    values = []
    
    def encode(row):
        result = []
        for base in row:
            result.append(bases[base])
            
        values.append(result)
        
    df["seq_new"].apply(encode)
    
    return values

In [7]:
neg_examples = transform(negative_with_seq)
pos_examples = transform(positive_with_seq)
features = np.concatenate((pos_examples, neg_examples), axis=0)
features[0][0]

array([1, 0, 0, 0, 0])

In [9]:
Y_pos = [[1]] * len(pos_examples)
Y_neg = [[0]] * len(neg_examples)
labels = np.concatenate((Y_pos, Y_neg), axis=0)


In [10]:
features.shape


(166348, 400, 5)

In [11]:
features_reshaped = features.reshape([166348, 2000])



In [12]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(features_reshaped,
                                                    labels,
                                                    test_size=0.33,
                                                    random_state=42)


In [14]:
# test it with linear regression

from sklearn.linear_model import LinearRegression

lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
# display all scores in one go

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", round(scores.mean()))
    print("Standard deviation:", scores.std())

In [19]:
from sklearn.model_selection import cross_val_score

lr_scores = cross_val_score(lr_clf, X_train, y_train, scoring="neg_mean_squared_error", cv=10)
lr_rmse_scores = np.sqrt(-lr_scores)
display_scores(lr_rmse_scores)

Scores: [  2.58736921e-01   4.31096977e+09   3.37093357e+09   1.11212652e+09
   4.42473422e+08   1.83545108e+09   2.63485935e+09   1.76787665e+07
   1.34292322e+09   5.26210346e+09]
Mean: 2032951916.0
Standard deviation: 1727894073.6


In [ ]:
from sklearn.metrics import mean_squared_error

lr_predictions = lr_clf.predict(X_train)
lr_mse = mean_squared_error(y_train, lr_predictions)
lr_smre = np.sqrt(lr_mse)
lr_smre


In [40]:
# predict for test with linear regression

y_predictions = lr_clf.predict(X_test)
y_test_unidimensional = np.reshape(y_test, -1)
y_predictions_unidimensional = np.reshape(y_predictions, -1)
linear_results = pd.DataFrame(
    {
        "predictions": y_predictions_unidimensional,
        "truth": y_test_unidimensional
    }
)
linear_results.to_csv("./predictions/linear_results.csv", index=False)

In [33]:
da = np.array([[1],[2]])
da

array([[1],
       [2]])

In [36]:
pd.DataFrame(
    {
        "truth": np.reshape(da, -1)
    }
)

,truth
0,1
1,2


In [34]:
np.reshape(da, -1)

array([1, 2])

In [39]:
y_predictions[0]

array([ 0.96069336])